In [1]:
import os
from pathlib import Path
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.float_format', lambda x: f'{x:.3f}')

In [3]:
data_path = Path('../../data/silver/dados_limpos.parquet').resolve()
if not data_path.exists():
    raise FileNotFoundError(f'Silver parquet not found at {data_path}. Run Bronze and Silver first.')

df = pd.read_parquet(data_path)
print(f'Dados limpos carregados de {data_path}. Shape: {df.shape}')

Dados limpos carregados de /home/jovyan/work/data/silver/dados_limpos.parquet. Shape: (28908, 33)


In [4]:
print('''
1. COMPLETUDE DOS DADOS''')
total_cells = df.shape[0] * df.shape[1]
filled_cells = df.count().sum()
completude_geral = (filled_cells / total_cells) * 100
print(f'Completude Geral: {completude_geral:.2f}%')

print('''
Completude por coluna:''')
for coluna in df.columns:
    percentual = (df[coluna].count() / len(df)) * 100
    print(f' - {coluna}: {percentual:.2f}%')


1. COMPLETUDE DOS DADOS
Completude Geral: 100.00%

Completude por coluna:
 - year: 100.00%
 - state: 100.00%
 - district: 100.00%
 - crop: 100.00%
 - season: 100.00%
 - data_ingestao: 100.00%
 - area: 100.00%
 - production: 100.00%
 - yield: 100.00%
 - jan: 100.00%
 - feb: 100.00%
 - mar: 100.00%
 - apr: 100.00%
 - may: 100.00%
 - jun: 100.00%
 - jul: 100.00%
 - aug: 100.00%
 - sep: 100.00%
 - oct: 100.00%
 - nov: 100.00%
 - dec: 100.00%
 - ann: 100.00%
 - jan_feb: 100.00%
 - mar_may: 100.00%
 - jun_sep: 100.00%
 - oct_dec: 100.00%
 - temp_annual: 100.00%
 - temp_jan_feb: 100.00%
 - temp_mar_may: 100.00%
 - temp_jun_sep: 100.00%
 - temp_oct_dec: 100.00%
 - state_district: 100.00%
 - data_processamento: 100.00%


In [5]:
print('''
2. UNICIDADE DOS DADOS''')
duplicatas = df.duplicated().sum()
unicidade = ((len(df) - duplicatas) / len(df)) * 100
print(f'Linhas unicas: {unicidade:.2f}%')
print(f'Duplicatas encontradas: {duplicatas}')


2. UNICIDADE DOS DADOS
Linhas unicas: 100.00%
Duplicatas encontradas: 0


In [6]:
print('''
3. VALORES INVALIDOS''')
for coluna in ['production', 'yield']:
    if coluna in df.columns:
        negativos = (df[coluna] < 0).sum()
        print(f' - {coluna}: {negativos} valores negativos (esperado: 0)')

ano_minimo = 1950
ano_atual = datetime.now().year
if 'year' in df.columns:
    fora_range_ano = ((df['year'] < ano_minimo) | (df['year'] > ano_atual)).sum()
    print(f' - year: {fora_range_ano} valores fora do range {ano_minimo}-{ano_atual}')


3. VALORES INVALIDOS
 - production: 0 valores negativos (esperado: 0)
 - yield: 0 valores negativos (esperado: 0)
 - year: 0 valores fora do range 1950-2025


In [7]:
print('''
4. VALIDADE DE DOMINIOS''')
culturas_validas = ['MAIZE', 'RICE', 'WHEAT', 'BARLEY']
if 'crop' in df.columns:
    culturas_invalidas = df[~df['crop'].isin(culturas_validas)]['crop'].nunique()
    print(f' - crop: {culturas_invalidas} culturas fora da lista esperada')

colunas_clima = ['annual', 'jun_sep', 'temp_jan_feb', 'temp_mar_may', 'temp_jun_sep', 'temp_oct_dec']
for coluna in colunas_clima:
    if coluna in df.columns:
        if 'temp_' not in coluna and (df[coluna] > 10000).any():
            print(f' - {coluna}: {(df[coluna] > 10000).sum()} valores de precipitacao > 10000')
        if 'temp_' in coluna and ((df[coluna] > 50) | (df[coluna] < -20)).any():
            print(f' - {coluna}: {((df[coluna] > 50) | (df[coluna] < -20)).sum()} temperaturas fora do range -20 a 50')


4. VALIDADE DE DOMINIOS
 - crop: 0 culturas fora da lista esperada


In [8]:
print('''
5. VISUALIZACOES''')
os.makedirs('../../reports', exist_ok=True)

try:
    plt.figure(figsize=(12, 7))
    completude_por_coluna = (df.count() / len(df) * 100).sort_values()
    completude_por_coluna.plot(kind='barh', color='skyblue')
    plt.xlabel('Completude (%)')
    plt.title('Completude dos Dados por Coluna')
    plt.xlim(0, 100)
    plt.grid(axis='x', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig('../../reports/quality_report_completude.png')
    plt.close()
    print(' - Grafico de completude salvo em ../../reports/quality_report_completude.png')

    y_jitter = np.random.normal(0, 0.05, size=len(df))

    if 'yield' in df.columns:
        plt.figure(figsize=(10, 6))
        plt.scatter(df['yield'], y_jitter, s=5, alpha=0.5, color='darkgreen')
        plt.title('Distribuicao de Yield (jitter)')
        plt.xlabel('yield')
        plt.ylabel('densidade (jitter)')
        plt.yticks([])
        plt.tight_layout()
        plt.savefig('../../reports/quality_report_yield_distribution.png')
        plt.close()
        print(' - Grafico de yield salvo em ../../reports/quality_report_yield_distribution.png')

    if 'production' in df.columns:
        plt.figure(figsize=(10, 6))
        plt.scatter(df['production'], y_jitter, s=5, alpha=0.5, color='darkred')
        plt.title('Distribuicao de Production (jitter)')
        plt.xlabel('production')
        plt.ylabel('densidade (jitter)')
        plt.yticks([])
        plt.ylim(-0.1, 0.1)
        plt.tight_layout()
        plt.savefig('../../reports/quality_report_production_distribution.png')
        plt.close()
        print(' - Grafico de production salvo em ../../reports/quality_report_production_distribution.png')

except Exception as exc:  # noqa: BLE001
    print(f'Aviso: nao foi possivel gerar graficos. Erro: {exc}')


5. VISUALIZACOES
 - Grafico de completude salvo em ../../reports/quality_report_completude.png
 - Grafico de yield salvo em ../../reports/quality_report_yield_distribution.png
 - Grafico de production salvo em ../../reports/quality_report_production_distribution.png


In [9]:
score_final = (completude_geral + unicidade) / 2
print(f'''
Score Final: {score_final:.2f}%''')

if score_final >= 95:
    print('Classificacao: EXCELENTE')
elif score_final >= 85:
    print('Classificacao: BOM')
elif score_final >= 75:
    print('Classificacao: REGULAR')
else:
    print('Classificacao: NECESSITA MELHORIAS')


Score Final: 100.00%
Classificacao: EXCELENTE
